In [ ]:
# Max Gadeberg
# Aug 29, 2024

import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Function to retrieve data
def fetch_data(url, user, pwd):
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    response = requests.get(url, auth=(user, pwd), headers=headers)
    
    if response.status_code != 200:
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
        return None
    return response.json()

incidents = '300'

# Set the request parameters
url = f'https://noanetdev.service-now.com/api/now/table/incident?sysparm_limit={incidents}&sysparm_query=ORDERBYDESCsys_created_on&sysparm_fields=number,configuration_item,u_choice_1,priority,party_responsible_for_outage,u_nn_complianceissue,u_next_action,short_description,event_summary,channel,opened_at,category,subcategory,assignment_group,assigned_to,work_start,work_end,u_customer,u_provider,location'

# Credentials
user = ''
pwd = ''

# Fetch data
data = fetch_data(url, user, pwd)
if not data:
    exit()

# Convert data to DataFrame
df = pd.DataFrame(data['result'])

# Ensure 'opened_at' is in datetime format
df['opened_at'] = pd.to_datetime(df['opened_at'], errors='coerce')

# Use all data (no date filtering)
df_filtered = df.copy()  # Make a copy to avoid SettingWithCopyWarning
print(df_filtered)

# Use .loc to avoid SettingWithCopyWarning
df_filtered.loc[:, 'date'] = df_filtered['opened_at'].dt.date

# Debug: Print unique dates and categories
print("Unique Dates:", df_filtered['date'].unique())
print("Unique Categories:", df_filtered['category'].unique())

# Define the mapping for category codes
category_mapping = {
    '1': 'Application',
    '2': 'Customer facing',
    '3': 'Equipment',
    '4': 'Network',
    '5': 'Fiber Break',
    '6': 'Power',
    '7': 'Other'
}

# Check if 'category' column is available
if 'category' not in df_filtered.columns:
    print("Category column is missing from the data")
else:
    # Replace numerical codes with descriptive names
    df_filtered['category'] = df_filtered['category'].astype(str).map(category_mapping).fillna('Unknown')

    # Group data by date and category, then count incidents
    grouped = df_filtered.groupby(['date', 'category']).size().reset_index(name='count')

    # Pivot the table to have dates as rows and categories as columns
    pivot_table = grouped.pivot(index='date', columns='category', values='count').fillna(0)

    # Plotting
    plt.figure(figsize=(12, 8))
    pivot_table.plot(kind='bar', stacked=True, colormap='tab20', figsize=(12, 8))
    plt.title('Number of Incidents per Category')
    plt.xlabel('Date')
    plt.ylabel('Number of Incidents')
    plt.xticks(rotation=45)
    plt.legend(title='Category')
    plt.tight_layout()
    plt.show()
